In [4]:
import pandas as pd
import re

# Load both sheets
df = pd.read_excel("DA - Task 1..xlsx", sheet_name="Task")
taxonomy = pd.read_excel("DA - Task 1..xlsx", sheet_name="Taxonomy")

# Clean up column names to remove spaces
taxonomy.columns = taxonomy.columns.str.strip()

# Clean your free-text fields
def clean_text(text):
    if pd.isnull(text): return ""
    return re.sub(r'[^\w\s]', '', text.lower().strip())

df["Complaint_Clean"] = df["Complaint"].apply(clean_text)
df["Cause_Clean"] = df["Cause"].apply(clean_text)
df["Correction_Clean"] = df["Correction"].apply(clean_text)

# Match taxonomy
def match_taxonomy(text, options):
    for term in options:
        if str(term).lower() in text:
            return term
    return ""

# Extract taxonomy lists
symptom_cond = taxonomy["Symptom Condition"].dropna().tolist()
symptom_comp = taxonomy["Symptom Component"].dropna().tolist()
root_causes = taxonomy["Root Cause"].dropna().tolist()
fix_cond = taxonomy["Fix Condition"].dropna().tolist()
fix_comp = taxonomy["Fix Component"].dropna().tolist()

# Apply matching
df["Symptom Condition"] = df["Complaint_Clean"].apply(lambda x: match_taxonomy(x, symptom_cond))
df["Symptom Component"] = df["Complaint_Clean"].apply(lambda x: match_taxonomy(x, symptom_comp))
df["Root Cause"] = df["Cause_Clean"].apply(lambda x: match_taxonomy(x, root_causes))
df["Fix Condition"] = df["Correction_Clean"].apply(lambda x: match_taxonomy(x, fix_cond))
df["Fix Component"] = df["Correction_Clean"].apply(lambda x: match_taxonomy(x, fix_comp))

# Save output
df.to_excel("Task1_Tagged_Output.xlsx", index=False)
